# Ceneo Scraper

## Struktura opinii w serwisie Ceneo.pl
|składowa|selektor|zmienna|
|--------|--------|-------|
| identyfikator opinii |["data-entry-id"]|opinion_id|
| autor |span.user-post__author-name|author|
| rekomendację |span.user-post__author-recomendation > em|recommed|
| liczbę gwiazdek |span.user-post__score-count|stars|
| treść opinii |div.user-post__text|content|
| listę wad |div.review-feature__item--negatives ~ div.review feature__item|cons|
| listę zalet |div.review-feature__item--positives ~ div.review feature__item|pros|
| data wystawienia opinii |span.user-post__published time:nth-child(1)["datetime"]|opinion_date|
| data zakupu produktu |span.user-post__published time:nth-child(2)["datetime"]|purchase_date|
| ile osób uznało opinię za przydatną |button.vote-yes["data-total-vote"]|up_vote|
| ile osób uznało opinię za nieprzydatną |button.vote-no["data-total-vote"]|down_vote|

## 1. Import bibliotek

In [1]:
import requests
from bs4 import BeautifulSoup

## Struktura opiniii w serwisie Ceneo.pl

## 2. Wczytanie kodu produktu o którym mają zostać pobrane dane

In [2]:
product_id="49551189"
url = f'https://www.ceneo.pl/{product_id}#tab=reviews_scroll'

## 3. wysyłanie do serwera Ceneo.pl żądania dostępu do strony z opiniami o produkcie

In [3]:
response = requests.get(url)
response.status_code

200

## 4. Przekształcenie kodu HTML w postaci tekstowej do obiektu reprezenującego strukturę DOM
## 5. Wydobycie ze struktury DOM znacznikow odpowiadajacych opinom konsumentów

In [4]:
page = BeautifulSoup(response.text, "html.parser")
opinions = page.select("div.js_product-review")
opinion = page.select_one("div.js_product-review")
print(type(page))
print(type(opinions))
print(type(opinion))

<class 'bs4.BeautifulSoup'>
<class 'bs4.element.ResultSet'>
<class 'bs4.element.Tag'>


## 6. Wydobycie ze struktury znacznika odpowiadającego pojedynczej opini jej składowych

In [39]:
def extract(ancestor,selector=None,attribute=None,return_list=False):
    if return_list:
        if attribute:
            return [tag[attribute].strip() for tag in ancestor.select(selector)]
        return [tag.text.strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            return ancestor.select_one(selector)[attribute].strip()
        return ancestor.select_one(selector).text.strip()
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.text.strip()

In [ ]:
single_opinion = {
    "opinion_id" : extract(opinion, None, "data-entry-id"),
    "author" : extract(opinion,"span.user-post__author-name"),
    "recommend" : extract(opinion, "span.user-post__author-recomendation > em"),
    "stars" : extract(opinion, "span.user-post__score-count"),
    "content" : extract(opinion, "div.user-post__text"),
    "cons" : extract(opinion,"div.review-feature__title--negatives ~ div.review-feature__item",None,True),
    "pros" : extract(opinion,"div.review-feature__title--positives ~ div.review-feature__item",None,True),
    "opinion_date" : extract(opinion, "span.user-post__published > time:nth-child(1)","datetime"),
    "purchase_date" : extract(opinion, "span.user-post__published > time:nth-child(2)","datetime"),
    "up_vote" : extract(opinion,"button.vote-yes","data-total-vote"),
    "down_vote" : extract(opinion,"button.vote-no","data-total-vote"),
}